In [38]:
%reset -f

In [39]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

In [46]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    print f
    print ddf[-1].shape
    
dfAll = pd.concat(ddf)
print dfAll.shape

results_pipe_test_0.3_train_0.2_Respiratory_last_all_readmisssion_vs_none_extended_none_none_roc_auc_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.2_Respiratory_last_all_readmisssion_vs_none_extended_none_none_recall_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.2_Respiratory_last_all_readmisssion_vs_none_extended_none_after_roc_auc_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.2_Respiratory_last_all_readmisssion_vs_none_extended_none_after_recall_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.4_Respiratory_last_all_readmisssion_vs_none_extended_none_none_roc_auc_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.4_Respiratory_last_all_readmisssion_vs_none_extended_none_none_recall_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.4_Respiratory_last_all_readmisssion_vs_none_extended_none_after_roc_auc_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.4_Respiratory_last_all_readmisssion_vs_none_extended_none_after_recall_logReg_.pkl
(1, 26)
results_pipe_test_0.3_train_0.6_

In [47]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_rec_mean = pd.to_numeric(dfAll.cv_rec_mean)
dfAll.cv_prec_mean = pd.to_numeric(dfAll.cv_prec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_prec_std = pd.to_numeric(dfAll.cv_prec_std)
dfAll.cv_rec_std = pd.to_numeric(dfAll.cv_rec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(2).astype(str).str.cat(dfAll["cv_f1_std"].round(2).astype(str), sep="+/-")
dfAll["cv_prec"] = dfAll["cv_prec_mean"].round(2).astype(str).str.cat(dfAll["cv_prec_std"].round(2).astype(str), sep="+/-")
dfAll["cv_rec"] = dfAll["cv_rec_mean"].round(2).astype(str).str.cat(dfAll["cv_rec_std"].round(2).astype(str), sep="+/-")
dfAll["num_nones"] = np.sum(np.hstack(((dfAll.sm == "none").reshape(-1,1),(dfAll.fs == "none").reshape(-1,1))), axis=1)

print dfAll.columns.tolist()
print dfAll.shape

['exp', 'typeDisease', 'typeEncounter', 'typeHypothesis', 'typeDataFeatures', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_f1', 'tr_prec', 'tr_rec', 'cv_f1_mean', 'cv_f1_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'test_f1', 'test_prec', 'test_rec', 'test_auc', 'time', 'pipeline', 'cv_f1', 'cv_prec', 'cv_rec', 'num_nones']
(11, 30)


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


In [48]:
dfAll.sort_values(["tr_f1","exp"], ascending=False,inplace=True)
dfAll[["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.20,none,roc_auc,logReg,"[balanced, 0.5, l2]",0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.59
0,0.40,none,roc_auc,logReg,"[balanced, 0.5, l2]",0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.59
0,0.60,none,roc_auc,logReg,"[balanced, 0.5, l2]",0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.59
0,0.20,after,roc_auc,logReg,"[None, 0.1, 4, l1]",0.61,0.62,0.60,0.61,0.62,0.60,0.59,0.59,0.60,0.59
0,0.40,after,roc_auc,logReg,"[None, 0.1, 4, l1]",0.61,0.62,0.60,0.61,0.62,0.60,0.59,0.59,0.60,0.59
0,0.60,after,roc_auc,logReg,"[None, 0.1, 4, l1]",0.61,0.62,0.60,0.61,0.62,0.60,0.59,0.59,0.60,0.59
0,0.20,none,recall,logReg,"[balanced, 0.005, l1]",0.57,0.59,0.56,0.56,0.59,0.57,0.56,0.55,0.58,0.56
0,0.40,none,recall,logReg,"[balanced, 0.005, l1]",0.57,0.59,0.56,0.56,0.59,0.57,0.56,0.55,0.58,0.56
0,0.60,none,recall,logReg,"[balanced, 0.005, l1]",0.57,0.59,0.56,0.56,0.59,0.57,0.56,0.55,0.58,0.56
0,0.20,after,recall,logReg,"[None, 0.005, 4, l1]",0.57,0.59,0.56,0.56,0.59,0.56,0.56,0.56,0.58,0.56


In [15]:
np.sum(dfs["time"])

693.3270304203033